In [11]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [12]:
data = pd.read_csv('BS_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price
0,RHI,2013-12-01,1490.3,570.6,919.6,135.3,6.80,718.3,5.31,1.4,-274.3,41.99
1,RCL,2011-12-01,19804.4,11396.6,8407.8,217.5,38.74,7442.4,34.29,8507.2,8245.0,24.77
2,NVDA,2020-01-01,17315.0,5111.0,12204.0,612.0,19.93,11537.0,18.84,2643.0,-8254.0,236.43
3,STC,2012-12-01,1291.2,710.8,580.4,21.1,29.31,263.5,13.58,71.2,-137.4,26.00
4,XRAY,2010-12-01,3258.0,1348.0,1909.9,142.1,12.98,457.6,3.23,786.6,246.6,34.17
5,GWW,2016-12-01,5694.3,3788.5,1905.8,58.8,30.57,684.7,11.64,2247.1,1972.9,232.25
6,SKM,2015-12-01,24288.4,11223.5,13064.9,70.6,183.55,9386.2,132.93,7039.9,5790.9,20.15
7,BAX,2011-12-01,19073.0,12245.0,6828.0,560.3,11.74,3442.0,6.14,5195.0,2290.0,26.88
8,MAR,2012-12-01,6342.0,7627.0,-1285.0,312.3,-4.13,-3274.0,-10.53,2935.0,2847.0,37.27
9,SJM,2019-04-01,16711.3,8740.8,7970.5,113.7,70.08,-5059.2,-44.48,5959.9,5858.6,122.63


In [13]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)
data

,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt
0,1490.3,570.6,919.6,135.3,6.80,718.3,5.31,1.4,-274.3
1,19804.4,11396.6,8407.8,217.5,38.74,7442.4,34.29,8507.2,8245.0
2,17315.0,5111.0,12204.0,612.0,19.93,11537.0,18.84,2643.0,-8254.0
3,1291.2,710.8,580.4,21.1,29.31,263.5,13.58,71.2,-137.4
4,3258.0,1348.0,1909.9,142.1,12.98,457.6,3.23,786.6,246.6
5,5694.3,3788.5,1905.8,58.8,30.57,684.7,11.64,2247.1,1972.9
6,24288.4,11223.5,13064.9,70.6,183.55,9386.2,132.93,7039.9,5790.9
7,19073.0,12245.0,6828.0,560.3,11.74,3442.0,6.14,5195.0,2290.0
8,6342.0,7627.0,-1285.0,312.3,-4.13,-3274.0,-10.53,2935.0,2847.0
9,16711.3,8740.8,7970.5,113.7,70.08,-5059.2,-44.48,5959.9,5858.6


In [14]:
col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done


,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price
0,0.001638,0.000683,0.028169,0.001691,0.000362,0.289974,0.001341,0.000003,0.185569,41.99
1,0.021935,0.013682,0.045186,0.002725,0.000484,0.305348,0.001492,0.017775,0.200646,24.77
2,0.019176,0.006134,0.053813,0.007687,0.000412,0.314710,0.001411,0.005522,0.171446,236.43
3,0.001418,0.000851,0.027398,0.000254,0.000448,0.288934,0.001384,0.000149,0.185811,26.00
4,0.003597,0.001616,0.030420,0.001776,0.000385,0.289378,0.001330,0.001644,0.186491,34.17
5,0.006298,0.004546,0.030410,0.000728,0.000453,0.289897,0.001374,0.004695,0.189546,232.25
6,0.026904,0.013474,0.055770,0.000877,0.001038,0.309792,0.002006,0.014709,0.196303,20.15
7,0.021124,0.014700,0.041596,0.007037,0.000380,0.296202,0.001345,0.010855,0.190107,26.88
8,0.007015,0.009155,0.023159,0.003917,0.000320,0.280846,0.001258,0.006132,0.191093,37.27
9,0.018507,0.010493,0.044193,0.001419,0.000604,0.276765,0.001081,0.012453,0.196423,122.63


In [15]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:9] # Parameters
y = NpMatrix[:,9] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[1.63843386e-03 6.82611560e-04 2.81693214e-02 ... 1.34060792e-03
  2.92521072e-06 1.85568632e-01]
 [2.19349988e-02 1.36816489e-02 4.51864494e-02 ... 1.49174313e-03
  1.77752519e-02 2.00645972e-01]
 [1.91761262e-02 6.13437854e-03 5.38134120e-02 ... 1.41116898e-03
  5.52237995e-03 1.71446269e-01]
 ...
 [6.79067888e-03 4.97604015e-03 3.06086506e-02 ... 1.34050362e-03
  4.01380699e-03 1.89340224e-01]
 [2.35835177e-03 1.36510305e-03 2.83540777e-02 ... 1.34947368e-03
  1.52340795e-03 1.86742708e-01]
 [7.21358632e-04 8.08087212e-04 2.60515545e-02 ... 1.31145520e-03
  1.21500717e-03 1.86722887e-01]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 9
Total Number of Training instances: 5000


[ 41.99  24.77 236.43 ...  49.01   8.07  34.69]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 41.99
Total number of y values 5000


In [16]:
#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=9, activation='relu', input_shape=(number_of_features,)))
    
     #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    
    network.add(layers.Dense(units=4, activation='relu', input_shape=(number_of_features,)))
    
     #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    
    network.add(layers.Dense(units=3, activation='relu', input_shape=(number_of_features,)))
    
     #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [17]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [18]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 3s 813us/sample - loss: 66.1147 - mae: 66.1147 - mse: 21377.6309
Epoch 2/1000
4000/4000 [==============================] - 2s 507us/sample - loss: 55.4757 - mae: 55.4757 - mse: 19331.2871
Epoch 3/1000
4000/4000 [==============================] - 2s 440us/sample - loss: 54.1883 - mae: 54.1883 - mse: 19011.7441
Epoch 4/1000
4000/4000 [==============================] - 2s 536us/sample - loss: 53.8377 - mae: 53.8378 - mse: 19125.2402
Epoch 5/1000
4000/4000 [==============================] - 2s 487us/sample - loss: 53.1412 - mae: 53.1413 - mse: 18968.5996
Epoch 6/1000
4000/4000 [==============================] - 2s 432us/sample - loss: 53.4437 - mae: 53.4437 - mse: 18908.0684
Epoch 7/1000
4000/4000 [==============================] - 2s 512us/sample - loss: 53.6323 - mae: 53.6324 - mse: 19093.1777
Epoch 8/1000
4000/4000 [==============================] - 2s 446us/sample - loss: 52.7548 - mae: 52.7548 - mse: 19017

Epoch 67/1000
4000/4000 [==============================] - 2s 436us/sample - loss: 46.2224 - mae: 46.2224 - mse: 17741.5996
Epoch 68/1000
4000/4000 [==============================] - 2s 402us/sample - loss: 46.1875 - mae: 46.1875 - mse: 17752.2988
Epoch 69/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 46.1016 - mae: 46.1016 - mse: 17759.5645
Epoch 70/1000
4000/4000 [==============================] - 2s 434us/sample - loss: 45.8599 - mae: 45.8599 - mse: 17701.9941
Epoch 71/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 46.3786 - mae: 46.3786 - mse: 17711.6406
Epoch 72/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 45.9111 - mae: 45.9111 - mse: 17774.1680
Epoch 73/1000
4000/4000 [==============================] - 2s 458us/sample - loss: 45.9469 - mae: 45.9469 - mse: 17679.5273
Epoch 74/1000
4000/4000 [==============================] - 2s 399us/sample - loss: 45.6727 - mae: 45.6727 - mse: 17631.6504
Epoch 75

4000/4000 [==============================] - 3s 650us/sample - loss: 44.6579 - mae: 44.6579 - mse: 17128.4883
Epoch 133/1000
4000/4000 [==============================] - 3s 648us/sample - loss: 44.6569 - mae: 44.6569 - mse: 17272.8223
Epoch 134/1000
4000/4000 [==============================] - 2s 551us/sample - loss: 44.3978 - mae: 44.3977 - mse: 17086.5293
Epoch 135/1000
4000/4000 [==============================] - 2s 442us/sample - loss: 44.1662 - mae: 44.1662 - mse: 17081.5156
Epoch 136/1000
4000/4000 [==============================] - 2s 400us/sample - loss: 44.1990 - mae: 44.1991 - mse: 17016.0254
Epoch 137/1000
4000/4000 [==============================] - 2s 420us/sample - loss: 44.3145 - mae: 44.3145 - mse: 17055.5820
Epoch 138/1000
4000/4000 [==============================] - 2s 408us/sample - loss: 44.1856 - mae: 44.1856 - mse: 17040.1973
Epoch 139/1000
4000/4000 [==============================] - 2s 432us/sample - loss: 44.4847 - mae: 44.4847 - mse: 17151.5781
Epoch 140/1000


4000/4000 [==============================] - 2s 607us/sample - loss: 42.8983 - mae: 42.8983 - mse: 16356.5166
Epoch 198/1000
4000/4000 [==============================] - 3s 669us/sample - loss: 43.3998 - mae: 43.3998 - mse: 16661.5820
Epoch 199/1000
4000/4000 [==============================] - 2s 617us/sample - loss: 43.2532 - mae: 43.2532 - mse: 16576.9902
Epoch 200/1000
4000/4000 [==============================] - 3s 645us/sample - loss: 43.4766 - mae: 43.4766 - mse: 16853.7207
Epoch 201/1000
4000/4000 [==============================] - 2s 590us/sample - loss: 43.0718 - mae: 43.0718 - mse: 16300.4883
Epoch 202/1000
4000/4000 [==============================] - 2s 423us/sample - loss: 43.2558 - mae: 43.2558 - mse: 16642.8418
Epoch 203/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 43.1535 - mae: 43.1535 - mse: 16387.5879
Epoch 204/1000
4000/4000 [==============================] - 2s 417us/sample - loss: 43.0283 - mae: 43.0283 - mse: 16559.6621
Epoch 205/1000


4000/4000 [==============================] - 2s 610us/sample - loss: 42.2558 - mae: 42.2559 - mse: 16042.2324
Epoch 263/1000
4000/4000 [==============================] - 3s 654us/sample - loss: 42.3349 - mae: 42.3349 - mse: 15954.7969
Epoch 264/1000
4000/4000 [==============================] - 2s 598us/sample - loss: 42.4638 - mae: 42.4638 - mse: 16323.2578
Epoch 265/1000
4000/4000 [==============================] - 3s 673us/sample - loss: 42.2211 - mae: 42.2210 - mse: 15928.5957
Epoch 266/1000
4000/4000 [==============================] - 2s 623us/sample - loss: 41.9308 - mae: 41.9308 - mse: 16129.2715
Epoch 267/1000
4000/4000 [==============================] - 3s 654us/sample - loss: 42.1380 - mae: 42.1380 - mse: 15760.9629
Epoch 268/1000
4000/4000 [==============================] - 3s 628us/sample - loss: 42.2258 - mae: 42.2257 - mse: 15950.2090
Epoch 269/1000
4000/4000 [==============================] - 3s 651us/sample - loss: 41.7028 - mae: 41.7028 - mse: 15655.4385
Epoch 270/1000


...

4000/4000 [==============================] - 1s 347us/sample - loss: 44.7777 - mae: 44.7777 - mse: 15815.2217
Epoch 717/1000
4000/4000 [==============================] - 1s 354us/sample - loss: 44.4223 - mae: 44.4223 - mse: 15558.3262
Epoch 718/1000
4000/4000 [==============================] - 1s 372us/sample - loss: 44.9222 - mae: 44.9222 - mse: 16044.6895
Epoch 719/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 44.7677 - mae: 44.7677 - mse: 15908.3379
Epoch 720/1000
4000/4000 [==============================] - 1s 349us/sample - loss: 44.8868 - mae: 44.8868 - mse: 16154.6162
Epoch 721/1000
4000/4000 [==============================] - 1s 362us/sample - loss: 44.2565 - mae: 44.2565 - mse: 15749.8076
Epoch 722/1000
4000/4000 [==============================] - 1s 355us/sample - loss: 44.7129 - mae: 44.7129 - mse: 15962.4404
Epoch 723/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 45.3888 - mae: 45.3888 - mse: 16449.8555
Epoch 724/1000


4000/4000 [==============================] - 1s 357us/sample - loss: 44.5170 - mae: 44.5170 - mse: 15892.2510
Epoch 782/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 44.7519 - mae: 44.7518 - mse: 15800.4756
Epoch 783/1000
4000/4000 [==============================] - 1s 356us/sample - loss: 45.0124 - mae: 45.0124 - mse: 16197.5146
Epoch 784/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 44.4904 - mae: 44.4904 - mse: 15601.3018
Epoch 785/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 44.6888 - mae: 44.6887 - mse: 15935.6133
Epoch 786/1000
4000/4000 [==============================] - 1s 348us/sample - loss: 44.7654 - mae: 44.7654 - mse: 15692.9531
Epoch 787/1000
4000/4000 [==============================] - 1s 357us/sample - loss: 44.8375 - mae: 44.8375 - mse: 16106.0303
Epoch 788/1000
4000/4000 [==============================] - 1s 370us/sample - loss: 44.8113 - mae: 44.8114 - mse: 15994.5908
Epoch 789/1000


4000/4000 [==============================] - 1s 371us/sample - loss: 44.3680 - mae: 44.3680 - mse: 16013.9131
Epoch 847/1000
4000/4000 [==============================] - 1s 352us/sample - loss: 44.7142 - mae: 44.7143 - mse: 16126.5176
Epoch 848/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 44.6467 - mae: 44.6467 - mse: 15937.5605
Epoch 849/1000
4000/4000 [==============================] - 1s 351us/sample - loss: 44.8688 - mae: 44.8687 - mse: 16090.7939
Epoch 850/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 44.5727 - mae: 44.5727 - mse: 15573.5303
Epoch 851/1000
4000/4000 [==============================] - 1s 356us/sample - loss: 44.8257 - mae: 44.8257 - mse: 15842.0811
Epoch 852/1000
4000/4000 [==============================] - 2s 378us/sample - loss: 44.7301 - mae: 44.7300 - mse: 16073.0713
Epoch 853/1000
4000/4000 [==============================] - 1s 303us/sample - loss: 45.0459 - mae: 45.0459 - mse: 15863.9600
Epoch 854/1000


4000/4000 [==============================] - 1s 298us/sample - loss: 44.4133 - mae: 44.4133 - mse: 15458.0586
Epoch 912/1000
4000/4000 [==============================] - 1s 279us/sample - loss: 44.8200 - mae: 44.8200 - mse: 15817.5781
Epoch 913/1000
4000/4000 [==============================] - 2s 375us/sample - loss: 44.4246 - mae: 44.4246 - mse: 15684.8311
Epoch 914/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 44.9269 - mae: 44.9269 - mse: 16074.9258
Epoch 915/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 44.4739 - mae: 44.4739 - mse: 15683.4180
Epoch 916/1000
4000/4000 [==============================] - 1s 319us/sample - loss: 44.8649 - mae: 44.8649 - mse: 16053.1475
Epoch 917/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 45.0154 - mae: 45.0154 - mse: 16132.4912
Epoch 918/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 44.7658 - mae: 44.7658 - mse: 15551.1357
Epoch 919/1000


4000/4000 [==============================] - 1s 282us/sample - loss: 44.7411 - mae: 44.7411 - mse: 15835.4043
Epoch 976/1000
4000/4000 [==============================] - 1s 326us/sample - loss: 45.1187 - mae: 45.1187 - mse: 16206.3232
Epoch 977/1000
4000/4000 [==============================] - 1s 288us/sample - loss: 44.7452 - mae: 44.7452 - mse: 15855.2725
Epoch 978/1000
4000/4000 [==============================] - 1s 326us/sample - loss: 44.3365 - mae: 44.3365 - mse: 15954.8340
Epoch 979/1000
4000/4000 [==============================] - 1s 293us/sample - loss: 44.6686 - mae: 44.6685 - mse: 15823.3076
Epoch 980/1000
4000/4000 [==============================] - 1s 311us/sample - loss: 44.3793 - mae: 44.3794 - mse: 15692.5078
Epoch 981/1000
4000/4000 [==============================] - 1s 314us/sample - loss: 44.3896 - mae: 44.3895 - mse: 15674.5264
Epoch 982/1000
4000/4000 [==============================] - 1s 324us/sample - loss: 44.5309 - mae: 44.5309 - mse: 15466.2646
Epoch 983/1000


In [19]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-45.521644   -38.73688096 -44.18828775 -40.00159692 -43.1950503 ]
Mean MAE: -42.32869198703766
Standard Deviation of the mean: 2.558127973149439
